# Build Freamework: Cloning and Building Darknet


In [ ]:
# clone darknet 
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet 
!make

Using the below helper function to view images

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# Move files: Uploading Google Drive Files


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd ..
%cd content/darknet

# TRAINING:



In order to create a custom YOLOv4-TINY detector we will need the following:

*   Labeled Dataset
*   .cfg file
*   obj.data and obj.names files
*   train.txt file, test.txt


# Step 1: Gathering and Labeling a Custom Dataset
In order to create an object detector you need a labeled dataset, this may be the most important area to focus on in order to obtain high accuracy models 



# Step 2: Moving Your Custom Dataset Into Your Cloud VM
I recommend renaming the folder with your images and text files on your local machine to be called '**obj**' and then creating a .zip folder of the 'obj' folder. 

In [ ]:
# See if obj.zip & test.zip is in the folder
!ls /mydrive/sea_turtle

In [ ]:
# copy the .zip files into the root directory of cloud VM
!cp /mydrive/sea_turtle/obj.zip ../
!cp /mydrive/sea_turtle/test.zip ../

In [ ]:
# unzip the zip files and its contents should now be in /darknet/data/obj & data/test
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

# Step 3: Configuring Files for Training
This step involves properly configuring your custom .cfg file, obj.data, obj.names and train.txt file.


In [ ]:
# download yoloV4 cfg to google drive and change its name
!cp cfg/yolov4-tiny.cfg /mydrive/sea_turtle/yolov4-tiny.cfg

```
set batch=24, this means we will be using 24 images for every training step
set subdivisions=8, the batch will be divided by 8 to decrease GPU VRAM requirement
width= 416 (multiples of 32)
height= 416 (increasing,decreases speed)
maxbatches= # classes * 3000 (How long to train for)
steps=(80% of maxbatch & 90% of maxbatch)
filters=(# of classes + 5) * 3
for memory issues set random = 0 
```

In [ ]:
# upload the yolov4-tiny .cfg back to cloud VM from Google Drive
!cp /mydrive/sea_turtle/yolov4-tiny.cfg ./cfg

Uploading obj.names & obj.data into darknet data folder

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/sea_turtle/obj.names ./data
!cp /mydrive/sea_turtle/obj.data  ./data

Uploading scripts to create a txt file of the train.zip & test.zip

In [ ]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /mydrive/sea_turtle/generate_train.py ./
!cp /mydrive/sea_turtle/generate_test.py ./

Now run the python script 

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify train.txt and text.txt can be seen in our darknet/data folder
!ls data/

# Step 4: Download pre-trained weights for the convolutional layers.
Using pretrained weights allows a more accurate detection with less training time

In [ ]:
# upload pretrained convolutional layer weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

#Hint:

To avoid colab logging you out hit (CTRL + SHIFT + i) to open consol 

Paste the following code into your console window and hit enter
```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

# Step 5: Train the model




In [ ]:
# cd back into the darknet folder to run detections
%cd ..
%cd /content/darknet

In [ ]:
# train your yolov4-tiny detector
#%%capture
!./darknet detector train data/obj.data cfg/yolov4-tiny.cfg yolov4-tiny.weights -dont_show 

Start training again from previous weights


In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-tiny.cfg /mydrive/sea_turtle/backup/backup/yolov4-tiny_2000.weights -dont_show

# Step 6: Checking the Mean Average Precision (mAP) of Your Model

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny.cfg /mydrive/sea_turtle/backup/backup/yolov4-tiny_3000.weights

# Step 7: Run Your Custom Object Detector!!!
You have done it! You now have a custom object detector to make your very own detections. Time to test it out and have some fun!

In [ ]:
# need to set our custom cfg to test mode 
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov4-tiny.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-tiny.cfg


In [ ]:
%cd ..
%cd /content/darknet

In [ ]:
# run your detector on google drive video
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show /mydrive/sea_turtle/video/DJI_0119.mov -i 0 -out_filename /mydrive/sea_turtle/results.JPG

In [ ]:
# run your custom detector on google drive image 
!./darknet detector test data/obj.data cfg/yolov4-tiny.cfg /mydrive/sea_turtle/backup/backup/yolov4-tiny_1000.weights /mydrive/sea_turtle/test/Hawksbill_Track_141.JPG -Threshold 
imShow('predictions.jpg')

# Cheers!